In [23]:
import logging
import warnings

import numpy as np
import pandas as pd

import helpers.hdbg as hdbg
import research_amp.soccer_prediction.models as rasoprmo
import research_amp.soccer_prediction.preproccesing as rasoprpr
import research_amp.soccer_prediction.utils as rasoprut
from scipy.special import factorial
from scipy.optimize import minimize


pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

In [24]:
dataframes = rasoprut.load_data_to_dataframe("datasets/")

In [10]:
data = dataframes["ISDBv2_df"]


In [12]:
# Convert date column to datetime
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
# Define half-life period (e.g., 180 days)
half_life_period = 180
# Calculate the age of each match in days (relative to the most recent match)
data['Days_Ago'] = (data['Date'].max() - data['Date']).dt.days
# Calculate time weights
data['Time_Weight'] = 0.5 ** (data['Days_Ago'] / half_life_period)
# Generate unique team identifiers.
teams = pd.Series(data['HT'].tolist() + data['AT'].tolist()).unique()
team_to_id = {team: idx for idx, team in enumerate(teams)}
# Map teams to unique identifiers.
data['HT_id'] = data['HT'].map(team_to_id)
data['AT_id'] = data['AT'].map(team_to_id)
# Display the first few rows of the dataset
print(data.head())

     Sea   Lge       Date              HT             AT  HS  AS  GD WDL  \
0  00-01  GER1 2000-08-11        Dortmund  Hansa Rostock   1   0   1   W   
1  00-01  GER1 2000-08-12   Bayern Munich  Hertha Berlin   4   1   3   W   
2  00-01  GER1 2000-08-12        Freiburg  VfB Stuttgart   4   0   4   W   
3  00-01  GER1 2000-08-12    Hamburger SV    Munich 1860   2   2   0   D   
4  00-01  GER1 2000-08-12  Kaiserslautern         Bochum   0   1  -1   L   

   Days_Ago   Time_Weight  HT_id  AT_id  
0      6165  4.894661e-11      0     17  
1      6164  4.913546e-11      1     12  
2      6164  4.913546e-11      2     13  
3      6164  4.913546e-11      3     10  
4      6164  4.913546e-11      4     11  


In [25]:
def bivariate_poisson_log_likelihood(params, data):
    c, h, rho, *strengths = params
    log_likelihood = 0 
    for _, row in data.iterrows():
        i, j, goals_i, goals_j, time_weight = row['HT_id'], row['AT_id'], row['HS'], row['AS'], row['Time_Weight']
        lambda_i = np.exp(c + strengths[i] + h - strengths[j])
        lambda_j = np.exp(c + strengths[j] - strengths[i] - h)
        cov = rho * np.sqrt(lambda_i * lambda_j)
        # Calculate joint probability
        joint_prob = 0
        for k in range(min(goals_i, goals_j) + 1):
            joint_prob += (
                (np.exp(-lambda_i - lambda_j - cov) * lambda_i**goals_i * lambda_j**goals_j * cov**k) /
                (factorial(goals_i) * factorial(goals_j) * factorial(k))
            )
        log_likelihood += time_weight * np.log(joint_prob)
    return -log_likelihood 

In [26]:
# Number of teams
num_teams = len(teams)
# Initial parameters: [c, h, rho, *strengths]
initial_params = [0, 0, 0.1] + [1] * num_teams

In [31]:
# Select the data for the league and season.
final_data = data[(data['Lge'] == 'ENG5') & 
                     ((data['Sea'] == '07-08') | 
                      (data['Sea'] == '06-07') | 
                      (data['Sea'] == '08-09'))]
# Set optimization options.
options = {
    'maxiter': 10,  
    'disp': True      
}
# Optimize parameters using the BFGS algorithm with options.
result = minimize(bivariate_poisson_log_likelihood, initial_params, args=(final_data.iloc[:552],), method='L-BFGS-B', options=options)
optimized_params = result.x
print("Optimized Parameters:", optimized_params)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         1471     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.60282D-04    |proj g|=  1.18502D-04
Optimized Parameters: [0.  0.  0.1 ... 1.  1.  1. ]



 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 1471      1     21      1     0     0   1.185D-04   6.603D-04
  F =   6.6028153290966245E-004

ABNORMAL_TERMINATION_IN_LNSRCH                              
